In [3]:
import os
import csv
import json
import pickle
import requests
from time import sleep

Consumer Key	XZoHfEhHTquhRinLcNrS
Consumer Secret	QozVgMYFgZkYwcCdXNOhApHklUlVKUxe
Request Token URL	https://api.discogs.com/oauth/request_token
Authorize URL	https://www.discogs.com/oauth/authorize
Access Token URL	https://api.discogs.com/oauth/access_token

In [4]:
key = 'XZoHfEhHTquhRinLcNrS'
secret = 'QozVgMYFgZkYwcCdXNOhApHklUlVKUxe'

### Get release thumbs by artist (much faster)

In [57]:
release_info = {}

In [58]:
artist_id = 1 #1
save_every = 10
directory = 'thumbs2' ###################

In [59]:
releases = set([_[6:-4] for _ in os.listdir(directory)])

In [ ]:
while True:
    print(len(releases), end='\r')
    if artist_id % save_every == 0:
        pickle.dump(release_info, open('release_info.p', 'wb'))

    while True:
        sleep(1)
        url = f'https://api.discogs.com/artists/{artist_id}/releases?key={key}&secret={secret}'
        r = requests.get(url, headers={'User-agent': 'FooBarApp/3.0'})
        if r.status_code == 200 or r.status_code == 404:
            break
    response = json.loads(r.text)
    if 'message' in response and response['message'] == 'Artist not found.':
        artist_id += 1
        continue

    for release in response.get('releases', []):
        image_url = release.get('thumb', '')

        id = release['main_release'] if release['type'] == 'master' else release['id']
        if image_url != '' and str(id) not in releases:
            artist = release.get('artist', '')
            title = release.get('title', '')
            release_info[str(id)] = (artist, title, image_url)

            while True:
                r = requests.get(image_url, headers={'User-agent': 'FooBarApp/3.0'})
                if r.status_code == 200 or r.status_code == 404:
                    break
            if r.status_code == 200:
                with open(f'{directory}/thumb_{id}.jpg', 'wb') as file:
                    file.write(r.content)
            releases.add(str(id))
                
    artist_id += 1

In [ ]:
pickle.dump(release_info, open('release_info.p', 'wb'))

In [40]:
artist_id

153329